<a href="https://colab.research.google.com/github/Hutaf/Resume-Matcher/blob/master/Resume_Matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resume matcher 
helps companies to find the ideal candidates based on job description

### Steps:
###### 1- Resume and job description parsing: convert pdf and docx files to list of string 
###### 2- Removing stop words and cleaning
###### 3- Use CountVector and cosine similarity metric to find the matching percentage 

## 1

In [ ]:
! pip install docx2txt
! pip install pdfminer3
! pip install PyPDF2
! pip install gensim
! pip install nltk

  Created wheel for docx2txt: filename=docx2txt-0.8-cp36-none-any.whl size=3963 sha256=95f72dd3585fd1da6fa04dc4d0471fcf16853b09d299f4c0f0f29a2b3a6d1c8c
  Stored in directory: /root/.cache/pip/wheels/b2/1f/26/a051209bbb77fc6bcfae2bb7e01fa0ff941b82292ab084d596
Successfully built docx2txt
     |████████████████████████████████| 5.0MB 2.8MB/s 
     |████████████████████████████████| 13.7MB 301kB/s 
  Created wheel for pdfminer3: filename=pdfminer3-2018.12.3.0-cp36-none-any.whl size=117806 sha256=6f8fab85c278d82c14a67ad05d855a0f37b8fb1244dde1d2b5e115bc9cb49ac1
  Stored in directory: /root/.cache/pip/wheels/27/35/61/7d4c0c350683751100186de869681db04d6f2c5ab603eb0510
Successfully built pdfminer3
     |████████████████████████████████| 81kB 2.3MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=2633f408f6925d87ca5ffba2271b83b6d3b94fd4ad9e0f15c488b23771ad0f4e
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a

In [ ]:
! pip install docx
! pip install python-docx

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for docx: filename=docx-0.2.4-cp36-none-any.whl size=53925 sha256=b83d4e08a894171506cc70a81fb3f63e3f0c7fb86332a210e0082867bfabf198
  Stored in directory: /root/.cache/pip/wheels/cc/8d/09/563edfd874a35c0c7ed129b6c4fa890efa4c26458bdec6ffc1
Successfully built docx
     |████████████████████████████████| 5.5MB 2.8MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=b00851b4532f42f67d8c74db171d80af12bdce113e9c209a6d5c958546201ceb
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Libraries
import os
import collections
import docx2txt
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import docx 

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def convert_pdf_to_txt(path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(path, 'rb') as fh:

        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()
    converter.close()
    fake_file_handle.close()
    return text

In [ ]:
projectDir = '/content/drive/My Drive/'
resumeDir = projectDir + 'Resumes/' 
jobPostDir= projectDir + 'Job/'

In [ ]:
# Extracting Resume and any file 

def MultipleConvert(FolderDir):
 resumeTxt = []
 for filename in os.listdir(FolderDir):
    if(filename.endswith(".pdf")):
        try:
            resumeTxt.append(convert_pdf_to_txt(FolderDir+filename))
        except Exception:
            print('Error reading .pdf file' + filename)
    if(filename.endswith(".docx")):
        try:
            doc = docx.Document(FolderDir + filename)
            data = ""
            fullText = []
            for para in doc.paragraphs:
                fullText.append(para.text)
                data = '\n'.join(fullText)
            resumeTxt.append(data)
        except IOError:
            print('Error reading .docx file' + filename)
    if(filename.endswith(".doc")):
        try:
            text = textract.process(FolderDir+filename).decode('utf-8')     
            resumeTxt.append(text)
        except Exception:
            print('Error reading .doc file :' + filename)
 return resumeTxt

In [ ]:
resumeTxt = MultipleConvert(resumeDir)
JobTxt = MultipleConvert(jobPostDir)

Error reading .doc file :Bangkok Bank_Mak Hwei Chin_Accountant.doc
Error reading .doc file :Gamevil_Belinda Zhang_Accounting Manager.doc
Error reading .doc file :Aviva_Elaine Cheah_Assistant FM.doc
Error reading .doc file :APL_Muthuraman_Enterprise Architect.doc
Error reading .doc file :Edrington_Lim Kim Yan_Finance Manager.doc
Error reading .doc file :Blackberry_Pradeep Ladd_Sr FM.doc
Error reading .doc file :Aberdeen_Magdalene Yow_Finance Manager.doc
Error reading .doc file :Trinitus AM_Vanessa Lam_Office Manager.doc
Error reading .doc file :Fidelity Management_Anna Chan_Equity Trading Coordinator.doc
Error reading .doc file :Sinopac AM_Charlotte Wong_EA.doc


In [ ]:
print(JobTxt[0])

CHIEF FINANCIAL OFFICER


JOB DESCRIPTION


Principal Accountabilities (Key Result Areas)

Oversee long-term budgetary planning and costs management in alignment with organization‟s strategic plan, especially as the organization considers sponsorships, potential acquisitions, and collaborations with external organizations.

Assess organizational performance against both the annual budget and long-term strategy. Develop tools and systems to provide critical financial and operational information to the management and make actionable recommendations on both strategy and operations.

Engage the board finance, audit, and investment committees around issues, trends, and changes in the operating model and operational delivery. Assist in establishing yearly objectives and meeting agendas, and selecting and engaging outside consultants (auditors, investment advisors).

Oversee all accounts, ledgers, and reporting systems, ensuring compliance with appropriate accounting principles, regulatory re

In [ ]:
print(resumeTxt[0])

Lim Hui Ling Dorothy
Phone: 81339491 Email: 

WORK EXPERIENCE

EQUINIX ASIA PACIFIC PTE LTD
General Ledger, Regional Shared Service Centre (Senior Accountant)
Summary of responsibilities:
Finance responsibilities:
Led the Shared Service Centre setup in 2014 for Hong Kong country (total – 2 entities and 1 branch office)
Perform and co-ordinate month-end closing with country team
Preparation, processing and posting of monthly journals
Processing of fixed assets, CIP% schedules
Preparation of monthly audit schedules
Preparation of monthly and quarterly fluctuation analysis 
Intercompany billing, reconciliation and confirmation
Review of lease agreements for operating, capital or built-to-suit leases, computing and assessing for asset retirement obligation
Preparation of quarterly accounting memos for assessment in compliance with US GAAP requirements
Preparation of monthly schedules for vendor information updates in system
Sign off for SOX internal controls review
Liaise with auditors for

In [ ]:
len(resumeTxt)

15

In [ ]:
len(JobTxt)

2

In [ ]:
type(resumeTxt)

list

In [ ]:
# NOT IMP
# Save Clean files into txt file
f=open('Clean_Resume.txt','w', encoding='utf-8')
for ele in resumeTxt:
    f.write(ele+'\n')
f.close()

f1=open('Clean_Job.txt','w', encoding='utf-8')
for ele in JobTxt:
    f1.write(ele+'\n')
f1.close()

## 2 

In [ ]:
from nltk.corpus import stopwords
stop_word = set(stopwords.words('english'))

In [ ]:
def Cleaning(listTxt):
 clean = []
 for x in listTxt:
    text = x
    text = text.split()
    text = [w for w in text if w not in stop_word] # Remove stop_words
    text = [w for w in text if w.isalpha()] # Remove numbers and special characters
    clean.append(" ".join(text))
 return clean

In [ ]:
# Clean resumes and job description
cleanresume = Cleaning(resumeTxt)
cleanjob = Cleaning(JobTxt)

In [ ]:
len(resumeTxt[0])

5160

In [ ]:
len(cleanresume[0])

3491

In [ ]:
len(JobTxt[0])

3786

In [ ]:
len(cleanjob[0])

2490

In [ ]:
cleanjob[0]

'CHIEF FINANCIAL OFFICER JOB DESCRIPTION Principal Accountabilities Result Oversee budgetary planning costs management alignment strategic especially organization considers potential collaborations external Assess organizational performance annual budget Develop tools systems provide critical financial operational information management make actionable recommendations strategy Engage board investment committees around changes operating model operational Assist establishing yearly objectives meeting selecting engaging outside consultants investment Oversee reporting ensuring compliance appropriate accounting regulatory budget audit Maintain internal control Manage cash flow Develop reliable cash flow projection process reporting mechanism includes minimum cash threshold meet operating Optimize handling banking relationships initiate appropriate strategies enhance cash Responsible get new working capital limit wherever required increase existing working capital limit smooth functioning O

In [ ]:
cleanresume[0]

'Lim Hui Ling Dorothy WORK EXPERIENCE EQUINIX ASIA PACIFIC PTE LTD General Regional Shared Service Centre Summary Finance Led Shared Service Centre setup Hong Kong country entities branch Perform closing country team processing posting monthly journals Processing fixed schedules Preparation monthly audit schedules Preparation monthly quarterly fluctuation analysis Intercompany reconciliation confirmation Review lease agreements capital computing assessing asset retirement obligation Preparation quarterly accounting memos assessment compliance US GAAP requirements Preparation monthly schedules vendor information updates system Sign SOX internal controls review Liaise auditors statutory group reporting requirements Finance Projects Involved User Acceptance Testing upgrade Oracle Accounting System identifying areas system improvement reports Involved REIT restructuring Hong Kong creation one additional legal entity User Acceptance Testing Oracle recording transfer entries Shadowing progra

## 3

just calculating cosine similarity between vectors

In [ ]:
job_description = str(cleanjob)

In [ ]:
type(job_description)

str

In [ ]:
# Extracting files names
resume_names = []
for filename in os.listdir(resumeDir):
    resume_names.append(os.path.splitext(filename)[0])
job_titles = []
for filename in os.listdir(jobPostDir):
    job_titles.append(os.path.splitext(filename)[0])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

def rank_cvs(clean_resume, job_description):
    match_percentages_list = []
    for i, x in enumerate(clean_resume):
        text = [x, job_description]
        count_matrix = cv.fit_transform(text)
        #get the match percentage
        matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
        match_percentages_list.append((i, round(matchPercentage, 3))) 
    for i, match_percentage in sorted(match_percentages_list, key=lambda x: -x[1]):
        print("\t("+  resume_names[i] +")"+ " resume matches about "+ str(match_percentage)+ "% of the job description.")
    

In [ ]:
import gensim.downloader as api

In [ ]:
model = api.load('word2vec-google-news-300')

[===-----------------------------------------------] 6.5% 107.9/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[====----------------------------------------------] 8.9% 148.0/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import scipy
def cosine_distance_wordembedding_method(s1, s2):
    m1 = []
    m2 = []
    for word in s1.split():
        try:
            m1.append(model[word])
        except KeyError:
            pass
    for word in s2.split():
        try:
            m2.append(model[word])
        except KeyError:
            pass
    vector_1 = np.mean(m1, axis=0)
    vector_2 = np.mean(m2,axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return round((1 - cosine)*100, 2)
    #print('Word Embedding method with a cosine distance asses that our two sentences are similar to',round((1-cosine)*100,2),'%')

In [ ]:
def rank_cvs_2(resume_list, job_list):
    for i, job in enumerate(job_list):
        rankings = [
                    (j, cosine_distance_wordembedding_method(resume, job))
                    for j, resume in enumerate(resume_list)
        ]
        print(f'{job_titles[i]}')
        for j, percent in sorted(rankings, key=lambda x: -x[1]):
            print(f'\t({resume_names[i]}) resume matches about {percent}% of the job description.')
        


In [ ]:
for i, job in enumerate(cleanjob):
  print(f'{job_titles[i]}')
  rank_cvs(cleanresume, job)

CHIEF FINANCIAL OFFICER
	(Dell_Manas Panda_Acc Advisor) resume matches about 32.185% of the job description.
	(Equinix_Dorothy Lim_Senior Accountant) resume matches about 31.124% of the job description.
	(Aviva_Elaine Cheah_Assistant FM) resume matches about 31.037% of the job description.
	(Gamevil_Belinda Zhang_Accounting Manager) resume matches about 26.034% of the job description.
	(GATES Construction_DEANNA SERENA_Finance Controller) resume matches about 25.427% of the job description.
	(APL_Muthuraman_Enterprise Architect) resume matches about 22.177% of the job description.
	(Blackberry_Pradeep Ladd_Sr FM) resume matches about 19.382% of the job description.
	(DNV GL_Novita Kie_FM) resume matches about 17.286% of the job description.
	(Edrington_Lim Kim Yan_Finance Manager) resume matches about 17.209% of the job description.
	(Bangkok Bank_Mak Hwei Chin_Accountant) resume matches about 16.771% of the job description.
	(Aberdeen_Magdalene Yow_Finance Manager) resume matches abou

In [ ]:
import numpy as np

In [ ]:
def rank_cvs_3(resume_list, job_list):
    for i, job in enumerate(job_list):
        rankings = [
                    (j, model.distance(resume, job))
                    for j, resume in enumerate(resume_list)
        ]
        print(f'{job_titles[i]}')
        for j, percent in sorted(rankings, key=lambda x: -x[1]):
            print(f'\t({resume_names[i]}) resume matches about {percent}% of the job description.')